In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime

In [7]:
def featurize(df, label):
  # Set label and features
  y = df[label]
  X = df.astype(float).assign(const=1)  # drop all categorical features and allow y-intercept to vary
  X = X.drop(columns=[label])
  return y, X

def mlr(df_pre_copy, label, print_results=True):
  df = df_pre_copy.copy()
  
  # Set label and features
  y = df[label]
  X = df.astype(float).assign(const=1)  # drop all categorical features and allow y-intercept to vary
  X = X.drop(columns=[label])

  # Run the multiple linear regression model
  model = sm.OLS(y, X)
  results = model.fit()

  # View results
  if(print_results): print(results.summary())
  return results

def results_summary(df, results, print_results=True):

  # Calculate Values
  r2 = results.rsquared
  r2_adj = results.rsquared_adj
  rmse = np.sqrt(sum((results.resid**2))/len(results.resid))
  mae = np.mean(abs(results.resid))
  mean = df['retweets'].mean()

  # Print Values
  if print_results:
    print(r2)
    print(r2_adj)
    print(rmse)
    print(mae)
    print(mean)
  
  return r2, r2_adj, rmse, mae, mean

In [8]:
def vif(df_pre_copy, label):
  df = df_pre_copy.copy()

  import pandas as pd
  from sklearn.linear_model import LinearRegression

  df.drop(inplace=True, columns=[label])

  # initialize dictionaries
  vif_dict, tolerance_dict = {}, {}

  try: df.drop(columns=['const'], inplace=True)
  except: pass

  # form input data for each exogenous variable
  for col in df:
    y = df[col]
    X = df.drop(columns=[col])
    
    # extract r-squared from the fit
    r_squared = LinearRegression().fit(X, y).score(X, y)

    # calculate VIF
    if r_squared < 1: # Prevent division by zero runtime error
      vif = 1/(1 - r_squared) 
    else:
      vif = 100
    vif_dict[col] = vif

    # calculate tolerance
    tolerance = 1 - r_squared
    tolerance_dict[col] = tolerance

    # generate the DataFrame to return
    df_output = pd.DataFrame({'VIF': vif_dict, 'Tolerance': tolerance_dict})

  return df_output.sort_values(by=['VIF'], ascending=False)

In [9]:
def remove_high_p(df, label, r2_diff_max=.01, print_results=False):
  r2_results = []
  df_copy = df.copy()  

  #Initial Test
  r2_results.append(mlr(df_copy, label, False))
  results = r2_results[-1]
  

  while (results.rsquared - results.rsquared_adj > r2_diff_max):
    drop_col = r2_results[-1].pvalues.sort_values(ascending=False).index[0]
    df_copy.drop(inplace=True, columns=[drop_col])
    r2_results.append(mlr(df_copy, label, False))
    results = r2_results[-1]

  df_copy[label] = df[label]
  return df_copy, r2_results

In [10]:
df = pd.read_csv("noReplies-clean.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'noReplies-clean.csv'

In [11]:
results = mlr(df, "interaction_score")
df, results_list = remove_high_p(df, "interaction_score")

NameError: name 'df' is not defined

In [12]:
results = mlr(df, "interaction_score")

NameError: name 'df' is not defined